# Partie 8 :

Partie : "To test this hypothesis, we generated adversarial examples on a deep maxout network and classified
 these examples using a shallow softmax network and a shallow RBF network. On examples that
 were misclassified by the maxout network, the RBF network predicted the maxout network’s class
 assignment only 16.0% of the time, while the softmax classifier predict the maxout network’s class
 correctly 54.6% of the time."

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np

# --- Hyperparameters ---
batch_size = 64
learning_rate = 0.001
momentum = 0.9
epsilon = 0.25
epochs = 10

# --- Load MNIST dataset ---
def load_data():
    transform = transforms.Compose([transforms.ToTensor()])
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

# --- Define Models ---
class MaxoutNetwork(nn.Module):
    def __init__(self, input_size=28*28, hidden_size=256, output_size=10):
        super(MaxoutNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size * 2)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.max(self.fc1(x).reshape(x.size(0), -1, 2), dim=2)[0]
        return self.fc2(x)

class SoftmaxNetwork(nn.Module):
    def __init__(self, input_size=28*28, output_size=10):
        super(SoftmaxNetwork, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.fc(x)

class RBFNetwork(nn.Module):
    def __init__(self, input_size=28*28, num_centers=10, output_size=10):
        super(RBFNetwork, self).__init__()
        self.centers = nn.Parameter(torch.randn(num_centers, input_size))
        self.beta = 1.0
        self.fc = nn.Linear(num_centers, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        distances = torch.cdist(x, self.centers)**2
        rbf_out = torch.exp(-self.beta * distances)
        return self.fc(rbf_out)

# --- FGSM Attack ---
def FGSM(model, images, labels, epsilon):
    images.requires_grad = True
    outputs = model(images)
    loss = nn.CrossEntropyLoss()(outputs, labels)
    model.zero_grad()
    loss.backward()
    perturbation = epsilon * images.grad.sign()
    adversarial_images = torch.clamp(images + perturbation, 0, 1)
    return adversarial_images

# --- Train Function ---
def train_model(model, optimizer, train_loader, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

# --- Evaluate Function ---
def evaluate_model(model, images, labels):
    model.eval()
    with torch.no_grad():
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
    return predictions

# --- Main Experiment ---
def main():
    # Load data
    train_loader, test_loader = load_data()

    # Initialize models
    maxout_model = MaxoutNetwork()
    softmax_model = SoftmaxNetwork()
    rbf_model = RBFNetwork()

    # Define optimizers
    maxout_optimizer = optim.SGD(maxout_model.parameters(), lr=learning_rate, momentum=momentum)
    softmax_optimizer = optim.SGD(softmax_model.parameters(), lr=learning_rate, momentum=momentum)
    rbf_optimizer = optim.SGD(rbf_model.parameters(), lr=learning_rate, momentum=momentum)

    # Train Maxout model
    print("\n--- Training Maxout Model ---")
    train_model(maxout_model, maxout_optimizer, train_loader, epochs)

    # Generate adversarial examples using Maxout model
    adversarial_images = []
    original_labels = []
    for images, labels in test_loader:
        adv_images = FGSM(maxout_model, images, labels, epsilon)
        adversarial_images.append(adv_images)
        original_labels.append(labels)

    adversarial_images = torch.cat(adversarial_images, dim=0)
    original_labels = torch.cat(original_labels, dim=0)

    # Train Softmax and RBF models
    print("\n--- Training Softmax Model ---")
    train_model(softmax_model, softmax_optimizer, train_loader, epochs)

    print("\n--- Training RBF Model ---")
    train_model(rbf_model, rbf_optimizer, train_loader, epochs)

    # Evaluate models on adversarial examples
    maxout_preds = evaluate_model(maxout_model, adversarial_images, original_labels)
    softmax_preds = evaluate_model(softmax_model, adversarial_images, original_labels)
    rbf_preds = evaluate_model(rbf_model, adversarial_images, original_labels)

    # Analyze results
    misclassified_mask = maxout_preds != original_labels
    softmax_agreement = (softmax_preds[misclassified_mask] == maxout_preds[misclassified_mask]).float().mean().item() * 100
    rbf_agreement = (rbf_preds[misclassified_mask] == maxout_preds[misclassified_mask]).float().mean().item() * 100

    print(f"Softmax agreement with Maxout on misclassified examples: {softmax_agreement:.2f}%")
    print(f"RBF agreement with Maxout on misclassified examples: {rbf_agreement:.2f}%")

if __name__ == "__main__":
    main()



--- Training Maxout Model ---
Epoch [1/10], Loss: 1.1204
Epoch [2/10], Loss: 0.4604
Epoch [3/10], Loss: 0.3654
Epoch [4/10], Loss: 0.3206
Epoch [5/10], Loss: 0.2904
Epoch [6/10], Loss: 0.2666
Epoch [7/10], Loss: 0.2467
Epoch [8/10], Loss: 0.2298
Epoch [9/10], Loss: 0.2152
Epoch [10/10], Loss: 0.2024

--- Training Softmax Model ---
Epoch [1/10], Loss: 1.0007
Epoch [2/10], Loss: 0.5524
Epoch [3/10], Loss: 0.4726
Epoch [4/10], Loss: 0.4336
Epoch [5/10], Loss: 0.4095
Epoch [6/10], Loss: 0.3927
Epoch [7/10], Loss: 0.3800
Epoch [8/10], Loss: 0.3698
Epoch [9/10], Loss: 0.3617
Epoch [10/10], Loss: 0.3548

--- Training RBF Model ---
Epoch [1/10], Loss: 2.3067
Epoch [2/10], Loss: 2.3020
Epoch [3/10], Loss: 2.3013
Epoch [4/10], Loss: 2.3012
Epoch [5/10], Loss: 2.3012
Epoch [6/10], Loss: 2.3012
Epoch [7/10], Loss: 2.3012
Epoch [8/10], Loss: 2.3012
Epoch [9/10], Loss: 2.3012
Epoch [10/10], Loss: 2.3012
Softmax agreement with Maxout on misclassified examples: 85.08%
RBF agreement with Maxout on mis

Le code suivant est un test pour la partie "These numbers are largely driven by the differing error rate of the
 different models though. If we exclude our attention to cases where both models being compared
 make a mistake, then softmax regression predict’s maxout’s class 84.6% of the time, while the RBF
 network is able to predict maxout’s class only 54.3% of the time. For comparison, the RBF network
 can predict softmax regression’s class 53.6% of the time, so it does have a strong linear component
 to its own behavior."

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np

# --- Hyperparameters ---
batch_size = 64
learning_rate = 0.001
momentum = 0.9
epsilon = 0.25
epochs = 10

# --- Load MNIST dataset ---
def load_data():
    transform = transforms.Compose([transforms.ToTensor()])
    train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

# --- Define Models ---
class MaxoutNetwork(nn.Module):
    def __init__(self, input_size=28*28, hidden_size=256, output_size=10):
        super(MaxoutNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size * 2)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.max(self.fc1(x).reshape(x.size(0), -1, 2), dim=2)[0]
        return self.fc2(x)

class SoftmaxNetwork(nn.Module):
    def __init__(self, input_size=28*28, output_size=10):
        super(SoftmaxNetwork, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.fc(x)

class RBFNetwork(nn.Module):
    def __init__(self, input_size=28*28, num_centers=10, output_size=10):
        super(RBFNetwork, self).__init__()
        self.centers = nn.Parameter(torch.randn(num_centers, input_size))
        self.beta = 1.0
        self.fc = nn.Linear(num_centers, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        distances = torch.cdist(x, self.centers)**2
        rbf_out = torch.exp(-self.beta * distances)
        return self.fc(rbf_out)

# --- FGSM Attack ---
def FGSM(model, images, labels, epsilon):
    images.requires_grad = True
    outputs = model(images)
    loss = nn.CrossEntropyLoss()(outputs, labels)
    model.zero_grad()
    loss.backward()
    perturbation = epsilon * images.grad.sign()
    adversarial_images = torch.clamp(images + perturbation, 0, 1)
    return adversarial_images

# --- Train Function ---
def train_model(model, optimizer, train_loader, epochs):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

# --- Evaluate Function ---
def evaluate_model(model, images, labels):
    model.eval()
    with torch.no_grad():
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
    return predictions

# --- Main Experiment ---
def main():
    # Load data
    train_loader, test_loader = load_data()

    # Initialize models
    maxout_model = MaxoutNetwork()
    softmax_model = SoftmaxNetwork()
    rbf_model = RBFNetwork()

    # Define optimizers
    maxout_optimizer = optim.SGD(maxout_model.parameters(), lr=learning_rate, momentum=momentum)
    softmax_optimizer = optim.SGD(softmax_model.parameters(), lr=learning_rate, momentum=momentum)
    rbf_optimizer = optim.SGD(rbf_model.parameters(), lr=learning_rate, momentum=momentum)

    # Train Maxout model
    print("\n--- Training Maxout Model ---")
    train_model(maxout_model, maxout_optimizer, train_loader, epochs)

    # Generate adversarial examples using Maxout model
    adversarial_images = []
    original_labels = []
    for images, labels in test_loader:
        adv_images = FGSM(maxout_model, images, labels, epsilon)
        adversarial_images.append(adv_images)
        original_labels.append(labels)

    adversarial_images = torch.cat(adversarial_images, dim=0)
    original_labels = torch.cat(original_labels, dim=0)

    # Train Softmax and RBF models
    print("\n--- Training Softmax Model ---")
    train_model(softmax_model, softmax_optimizer, train_loader, epochs)

    print("\n--- Training RBF Model ---")
    train_model(rbf_model, rbf_optimizer, train_loader, epochs)

    # Evaluate models on adversarial examples
    maxout_preds = evaluate_model(maxout_model, adversarial_images, original_labels)
    softmax_preds = evaluate_model(softmax_model, adversarial_images, original_labels)
    rbf_preds = evaluate_model(rbf_model, adversarial_images, original_labels)

    # Analyze results excluding cases where both models make mistakes
    misclassified_mask = maxout_preds != original_labels
    both_misclassified_mask = (softmax_preds[misclassified_mask] != original_labels[misclassified_mask]) & \
                              (rbf_preds[misclassified_mask] != original_labels[misclassified_mask])

    filtered_softmax_preds = softmax_preds[misclassified_mask][~both_misclassified_mask]
    filtered_maxout_preds = maxout_preds[misclassified_mask][~both_misclassified_mask]

    filtered_rbf_preds = rbf_preds[misclassified_mask][~both_misclassified_mask]
    filtered_softmax_agreement = (filtered_softmax_preds == filtered_maxout_preds).float().mean().item() * 100
    filtered_rbf_agreement = (filtered_rbf_preds == filtered_maxout_preds).float().mean().item() * 100

    # RBF agreement with Softmax
    filtered_rbf_softmax_agreement = (filtered_rbf_preds == filtered_softmax_preds).float().mean().item() * 100

    print(f"Softmax predicts Maxout's class (filtered): {filtered_softmax_agreement:.2f}%")
    print(f"RBF predicts Maxout's class (filtered): {filtered_rbf_agreement:.2f}%")
    print(f"RBF predicts Softmax's class (filtered): {filtered_rbf_softmax_agreement:.2f}%")

if __name__ == "__main__":
    main()



--- Training Maxout Model ---
Epoch [1/10], Loss: 1.1360
Epoch [2/10], Loss: 0.4665
Epoch [3/10], Loss: 0.3701
Epoch [4/10], Loss: 0.3241
Epoch [5/10], Loss: 0.2925
Epoch [6/10], Loss: 0.2680
Epoch [7/10], Loss: 0.2477
Epoch [8/10], Loss: 0.2307
Epoch [9/10], Loss: 0.2160
Epoch [10/10], Loss: 0.2035

--- Training Softmax Model ---
Epoch [1/10], Loss: 0.9826
Epoch [2/10], Loss: 0.5498
Epoch [3/10], Loss: 0.4710
Epoch [4/10], Loss: 0.4326
Epoch [5/10], Loss: 0.4086
Epoch [6/10], Loss: 0.3918
Epoch [7/10], Loss: 0.3793
Epoch [8/10], Loss: 0.3694
Epoch [9/10], Loss: 0.3613
Epoch [10/10], Loss: 0.3545

--- Training RBF Model ---
Epoch [1/10], Loss: 2.3081
Epoch [2/10], Loss: 2.3022
Epoch [3/10], Loss: 2.3013
Epoch [4/10], Loss: 2.3012
Epoch [5/10], Loss: 2.3012
Epoch [6/10], Loss: 2.3012
Epoch [7/10], Loss: 2.3012
Epoch [8/10], Loss: 2.3012
Epoch [9/10], Loss: 2.3012
Epoch [10/10], Loss: 2.3012
Softmax predicts Maxout's class (filtered): 66.60%
RBF predicts Maxout's class (filtered): 0.00%